<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Inferencias</h4>
</center>

In [1]:
# Importación librerias

# !pip install awswrangler
# !pip install boto3
import boto3
import awswrangler as wr
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [73]:
# Definición fechas

date_start=datetime.strptime('2017-01-01', '%Y-%m-%d')
date_end=datetime.strptime('2019-12-01', '%Y-%m-%d')
forecast_length = 2
date_future=date_end+relativedelta(months=forecast_length)
years=range(2018, 2020)

print(f"Fecha inicio historia {date_start}")
print(f"Fecha fin historia {date_end}")
print(f"Fecha futura (related time series) {date_future}")

# Versión modelo
data_version='4'
data_version_cluster1 = '4_c1'
data_version_cluster2 = '4_c2'
data_version_cluster3 = '4_c3'
grouped_columns = ["product_id"]
grouped_cols_name = "product"

# Parametros backtest
dataset_frequency='M'
quantiles=["mean"]

Fecha inicio historia 2017-01-01 00:00:00
Fecha fin historia 2019-12-01 00:00:00
Fecha futura (related time series) 2020-02-01 00:00:00


In [4]:
# Conexiones AWS

account_id=boto3.client('sts').get_caller_identity().get('Account')
session = boto3.Session() 
forecast = session.client(service_name='forecast')
region = forecast.meta.region_name
forecastquery = session.client(service_name='forecastquery')
role = f'arn:aws:iam::{account_id}:role/ForecastRole'

In [5]:
# Buckets de datos

# CLUSTER 1
target_s3_path_c1=f's3://datasets-forecast-account1/modelo-{data_version_cluster1}-forecast-{grouped_cols_name}/target/df_ventas_{grouped_cols_name}.csv'
items_s3_path_c1=f's3://datasets-forecast-account1/modelo-{data_version_cluster1}-forecast-{grouped_cols_name}/product/df_products.csv'
related_s3_path_c1=f's3://datasets-forecast-account1/modelo-{data_version_cluster1}-forecast-{grouped_cols_name}/related/df_related_{grouped_cols_name}.csv'

# CLUSTER 2
target_s3_path_c2=f's3://datasets-forecast-account1/modelo-{data_version_cluster2}-forecast-{grouped_cols_name}/target/df_ventas_{grouped_cols_name}.csv'
items_s3_path_c2=f's3://datasets-forecast-account1/modelo-{data_version_cluster2}-forecast-{grouped_cols_name}/product/df_products.csv'
related_s3_path_c2=f's3://datasets-forecast-account1/modelo-{data_version_cluster2}-forecast-{grouped_cols_name}/related/df_related_{grouped_cols_name}.csv'

# CLUSTER 3
target_s3_path_c3=f's3://datasets-forecast-account1/modelo-{data_version_cluster3}-forecast-{grouped_cols_name}/target/df_ventas_{grouped_cols_name}.csv'
items_s3_path_c3=f's3://datasets-forecast-account1/modelo-{data_version_cluster3}-forecast-{grouped_cols_name}/product/df_products.csv'
related_s3_path_c3=f's3://datasets-forecast-account1/modelo-{data_version_cluster3}-forecast-{grouped_cols_name}/related/df_related_{grouped_cols_name}.csv'

# Outputs
backtest_export_path_c1=f's3://datasets-forecast-account1/modelo-{data_version_cluster1}-forecast-{grouped_cols_name}/output/'
backtest_export_path_c2=f's3://datasets-forecast-account1/modelo-{data_version_cluster2}-forecast-{grouped_cols_name}/output/'
backtest_export_path_c3=f's3://datasets-forecast-account1/modelo-{data_version_cluster3}-forecast-{grouped_cols_name}/output/'

In [6]:
# Formar ARNs

# CLUSTER 1
dataset_group_arn_c1=f'arn:aws:forecast:us-east-1:{account_id}:dataset-group/modelo_{data_version_cluster1}_{grouped_cols_name}'
automl_predictor_arn_c1 = f'arn:aws:forecast:us-east-1:{account_id}:predictor/modelo_{data_version_cluster1}_predictor_01J02Q23DBHN6P2C8QM61MAY3F'

# CLUSTER 2
dataset_group_arn_c2=f'arn:aws:forecast:us-east-1:{account_id}:dataset-group/modelo_{data_version_cluster2}_{grouped_cols_name}'
automl_predictor_arn_c2 = f'arn:aws:forecast:us-east-1:{account_id}:predictor/modelo_{data_version_cluster2}_predictor_01J02Q4FXFQADEHVVGBN5ETE7C'

# CLUSTER 3
dataset_group_arn_c3=f'arn:aws:forecast:us-east-1:{account_id}:dataset-group/modelo_{data_version_cluster3}_{grouped_cols_name}'
automl_predictor_arn_c3 = f'arn:aws:forecast:us-east-1:{account_id}:predictor/modelo_{data_version_cluster3}_predictor_01J02Q5EW0HRWSQEVHF4M5B8WP'

In [7]:
# Verificar comunicación con Amazon Forecast
assert forecast.list_predictors()

In [8]:
best_quantile = ["mean"]

### Generar inferencias

- CLUSTER 1

In [10]:
forecast_name_c1=f"modelo_{data_version_cluster1}_{grouped_cols_name}_inference"
tags_c1=[{'Key': 'ProjectName', 'Value': 'pruebas_laboratorio'}]

response_c1 = forecast.create_forecast(
    ForecastName = forecast_name_c1,
    PredictorArn = automl_predictor_arn_c1,
    ForecastTypes= best_quantile,
    Tags=tags_c1
)

In [11]:
forecast_arn_c1=response_c1['ForecastArn']
forecast_arn_c1

'arn:aws:forecast:us-east-1:935182042847:forecast/modelo_4_c1_product_inference'

In [38]:
forecast.describe_forecast(
    ForecastArn=forecast_arn_c1
)['Status']

'ACTIVE'

- CLUSTER 2

In [13]:
forecast_name_c2=f"modelo_{data_version_cluster2}_{grouped_cols_name}_inference"
tags_c2=[{'Key': 'ProjectName', 'Value': 'pruebas_laboratorio'}]

response_c2 = forecast.create_forecast(
    ForecastName = forecast_name_c2,
    PredictorArn = automl_predictor_arn_c2,
    ForecastTypes= best_quantile,
    Tags=tags_c2
)

In [14]:
forecast_arn_c2=response_c2['ForecastArn']
forecast_arn_c2

'arn:aws:forecast:us-east-1:935182042847:forecast/modelo_4_c2_product_inference'

In [26]:
forecast.describe_forecast(
    ForecastArn=forecast_arn_c2
)['Status']

'ACTIVE'

- Cluster 3

In [16]:
forecast_name_c3=f"modelo_{data_version_cluster3}_{grouped_cols_name}_inference"
tags_c3=[{'Key': 'ProjectName', 'Value': 'pruebas_laboratorio'}]

response_c3 = forecast.create_forecast(
    ForecastName = forecast_name_c3,
    PredictorArn = automl_predictor_arn_c3,
    ForecastTypes= best_quantile,
    Tags=tags_c3
)

In [17]:
forecast_arn_c3=response_c3['ForecastArn']
forecast_arn_c3

'arn:aws:forecast:us-east-1:935182042847:forecast/modelo_4_c3_product_inference'

In [34]:
forecast.describe_forecast(
    ForecastArn=forecast_arn_c3
)['Status']

'ACTIVE'

### Exportar resultados

- Cluster 1

In [41]:
export_s3_path_c1=f's3://datasets-forecast-account1/modelo-{data_version_cluster1}-forecast-{grouped_cols_name}/inferencias/'
forecast_export_job_name_c1=f"modelo_{data_version_cluster1}_{grouped_cols_name}_inference_export"

In [42]:
response_c1 = forecast.create_forecast_export_job(
    ForecastExportJobName=forecast_export_job_name_c1,
    ForecastArn=forecast_arn_c1,
    Destination={
        'S3Config': {
            'Path': export_s3_path_c1,
            'RoleArn': role
        }
    },
    Tags=tags_c1
)

In [43]:
forecast_export_job_arn_c1=response_c1['ForecastExportJobArn']

In [53]:
forecast.describe_forecast_export_job(
    ForecastExportJobArn=forecast_export_job_arn_c1
)['Status']

'ACTIVE'

- Cluster 2

In [45]:
export_s3_path_c2=f's3://datasets-forecast-account1/modelo-{data_version_cluster2}-forecast-{grouped_cols_name}/inferencias/'
forecast_export_job_name_c2=f"modelo_{data_version_cluster2}_{grouped_cols_name}_inference_export"

In [46]:
response_c2 = forecast.create_forecast_export_job(
    ForecastExportJobName=forecast_export_job_name_c2,
    ForecastArn=forecast_arn_c2,
    Destination={
        'S3Config': {
            'Path': export_s3_path_c2,
            'RoleArn': role
        }
    },
    Tags=tags_c2
)

In [47]:
forecast_export_job_arn_c2=response_c2['ForecastExportJobArn']

In [58]:
forecast.describe_forecast_export_job(
    ForecastExportJobArn=forecast_export_job_arn_c2
)['Status']

'ACTIVE'

- Cluster 3

In [49]:
export_s3_path_c3=f's3://datasets-forecast-account1/modelo-{data_version_cluster3}-forecast-{grouped_cols_name}/inferencias/'
forecast_export_job_name_c3=f"modelo_{data_version_cluster3}_{grouped_cols_name}_inference_export"

In [50]:
response_c3 = forecast.create_forecast_export_job(
    ForecastExportJobName=forecast_export_job_name_c3,
    ForecastArn=forecast_arn_c3,
    Destination={
        'S3Config': {
            'Path': export_s3_path_c3,
            'RoleArn': role
        }
    },
    Tags=tags_c3
)

In [51]:
forecast_export_job_arn_c3=response_c3['ForecastExportJobArn']

In [59]:
forecast.describe_forecast_export_job(
    ForecastExportJobArn=forecast_export_job_arn_c3
)['Status']

'ACTIVE'

### Lectura de forecasts

In [65]:
df_predictions_c1=wr.s3.read_csv(export_s3_path_c1)
print("----------C1-----------")
print(df_predictions_c1.date.min(), df_predictions_c1.date.max())
print(len(df_predictions_c1.item_id.drop_duplicates()))
print(df_predictions_c1.head(1))

df_predictions_c2=wr.s3.read_csv(export_s3_path_c2)
print("----------C2-----------")
print(df_predictions_c2.date.min(), df_predictions_c2.date.max())
print(len(df_predictions_c2.item_id.drop_duplicates()))
print(df_predictions_c2.head(1))

df_predictions_c3=wr.s3.read_csv(export_s3_path_c3)
print("----------C3-----------")
print(df_predictions_c3.date.min(), df_predictions_c3.date.max())
print(len(df_predictions_c3.item_id.drop_duplicates()))
print(df_predictions_c3.head(1))

----------C1-----------
2020-01-01T00:00:00Z 2020-02-01T00:00:00Z
245
   item_id   brand   cat1      cat2     cat3  sku_size                  date   
0    20158  mayos3  foods  aderezos  ketchup       240  2020-01-01T00:00:00Z  \

        mean  
0  49.323593  
----------C2-----------
2020-01-01T00:00:00Z 2020-02-01T00:00:00Z
394
   item_id     brand cat1     cat2            cat3  sku_size   
0    20193  shampoo3   pc  cabello  acondicionador       350  \

                   date      mean  
0  2020-01-01T00:00:00Z  37.11784  
----------C3-----------
2020-01-01T00:00:00Z 2020-02-01T00:00:00Z
141
   item_id  brand cat1  cat2  cat3  sku_size                  date      mean
0    20569  nivea   pc  deos  aero        89  2020-01-01T00:00:00Z  9.519287


In [66]:
df_predictions=pd.concat([df_predictions_c1, df_predictions_c2, df_predictions_c3])

In [67]:
len(df_predictions.drop_duplicates(subset="item_id"))

780

In [68]:
for i in best_quantile:
    df_predictions[i]=np.where(df_predictions[i]<0, 0, df_predictions[i])
df_predictions.head(5)

,item_id,brand,cat1,cat2,cat3,sku_size,date,mean
0,20158,mayos3,foods,aderezos,ketchup,240,2020-01-01T00:00:00Z,49.323593
1,20158,mayos3,foods,aderezos,ketchup,240,2020-02-01T00:00:00Z,54.278352
2,21248,vichy,pc,cabello,shampoo,250,2020-01-01T00:00:00Z,0.015512
3,21248,vichy,pc,cabello,shampoo,250,2020-02-01T00:00:00Z,0.021669
4,21259,vichy,pc,cabello,shampoo,250,2020-01-01T00:00:00Z,0.012725


In [69]:
df_predictions.date.unique()

array(['2020-01-01T00:00:00Z', '2020-02-01T00:00:00Z'], dtype=object)

In [70]:
df_predictions = df_predictions[df_predictions.date == '2020-02-01T00:00:00Z']
print(df_predictions.date.unique())
df_predictions = df_predictions[["item_id", "mean"]]
df_predictions = df_predictions.rename(columns = {"item_id":"product_id", "mean":"tn"})
df_predictions.head(5)

['2020-02-01T00:00:00Z']


,product_id,tn
1,20158,54.278352
3,21248,0.021669
5,21259,0.012721
7,20622,5.028533
9,20107,84.616261


In [71]:
len(df_predictions.product_id.unique())

780

In [74]:
df_predictions["product_id"] = df_predictions["product_id"].astype(int)
df_predictions.to_csv(f"C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_modelo{data_version}_AWS.csv", sep = ",", index = False)

In [76]:
df_predictions.tn.sum()

27343.88922191103